# 🚀 Launcher – Financial-Tweet Agent
Instala dependencias, exporta claves opcionales, lanza Streamlit y crea un túnel ngrok.

In [ ]:
%%bash
if [ ! -f venv_installed.marker ]; then
  echo "🔧 Instalando dependencias…"
  pip install --upgrade pip
  pip install --no-cache-dir -r requirements.txt
  touch venv_installed.marker
else
  echo " Dependencias ya instaladas"
fi

In [ ]:
import os, getpass, subprocess, textwrap

def maybe_set(var: str, prompt: str):
    if not os.getenv(var):
        val = getpass.getpass(f"{prompt} (enter para omitir): ")
        if val:
            os.environ[var] = val

maybe_set("OPENAI_API_KEY", "OPENAI_API_KEY")
maybe_set("TWITTER_BEARER", "TWITTER_BEARER")

# Exportamos los que existan al entorno de bash (para Streamlit)
bash_exports = []
for var in ("OPENAI_API_KEY", "TWITTER_BEARER"):
    if os.getenv(var):
        bash_exports.append(f'export {var}="{os.environ[var]}"')

script = "\n".join(bash_exports)
if script:
    subprocess.run(["bash", "-c", script])
else:
    print(" Claves omitidas: el chat GPT o la búsqueda en vivo pueden mostrar advertencias.")

In [ ]:
%%bash --bg
pkill -f "streamlit run" || true
nohup streamlit run app.py \
      --server.headless true \
      --server.address 0.0.0.0 \
      --server.port 8501 \
      --server.enableCORS false \
      --server.enableXsrfProtection false \
      &> streamlit_logs.txt &

In [ ]:
from pyngrok import ngrok
import time, requests

ngrok.kill()
public = ngrok.connect(8501, "http")
print("Dashboard URL:", public.public_url)

# Health-check local
for _ in range(10):
    try:
        if requests.get("http://localhost:8501/_stcore/health").ok:
            print("✅ Streamlit vivo ✔️")
            break
    except requests.ConnectionError:
        time.sleep(1)
else:
    print("  No hubo respuesta en 10 s — revisa streamlit_logs.txt")

## ⏹ Cómo detener
```bash
pkill -f "streamlit run"
ngrok kill
```